In [2]:
from netCDF4 import Dataset as NetCDFFile
import numpy as np
from numpy import matlib
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.basemap import Basemap
from matplotlib import colors
import xarray as xr
from numpy import meshgrid
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.font_manager import findfont, FontProperties
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
font = findfont(FontProperties(family=['sans-serif']))
font

'/System/Library/Fonts/LucidaGrande.ttc'

## Ranking by Country

In [3]:
owda = xr.open_dataset('owda_hd_fix_500.nc').pdsi
import reverse_geocode
#Country Masks
UK_mask = np.zeros((114,88));France_mask = np.zeros((114,88));Germany_mask = np.zeros((114,88));
Switzerland_mask = np.zeros((114,88));Austria_mask = np.zeros((114,88));Belgium_mask = np.zeros((114,88));
Netherlands_mask = np.zeros((114,88));Luxembourg_mask = np.zeros((114,88));Italy_mask = np.zeros((114,88));
Europe_mask = np.zeros((114,88)); Europe_mask[:,:] = np.nan

for x in range(0,114):
    for y in range(0,88):
        coordinates = (owda.lat[y], owda.lon[x]), 
        if 'United Kingdom' in str(reverse_geocode.search(coordinates)):
            UK_mask[x,y] = 1
            Europe_mask[x,y] = 1
        else:
            UK_mask[x,y] = np.nan
            
        if 'France' in str(reverse_geocode.search(coordinates)):
            France_mask[x,y] = 1
            Europe_mask[x,y] = 1
        else:
            France_mask[x,y] = np.nan

        if 'Germany' in str(reverse_geocode.search(coordinates)):
            Germany_mask[x,y] = 1
            Europe_mask[x,y] = 1
        else:
            Germany_mask[x,y] = np.nan

        if 'Switzerland' in str(reverse_geocode.search(coordinates)):
            Switzerland_mask[x,y] = 1
        else:
            Switzerland_mask[x,y] = np.nan

        if 'Austria' in str(reverse_geocode.search(coordinates)):
            Austria_mask[x,y] = 1
            Europe_mask[x,y] = 1
        else:
            Austria_mask[x,y] = np.nan

        if 'Belgium' in str(reverse_geocode.search(coordinates)):
            Belgium_mask[x,y] = 1
            Europe_mask[x,y] = 1
        else:
            Belgium_mask[x,y] = np.nan
            
        if 'Netherlands' in str(reverse_geocode.search(coordinates)):
            Netherlands_mask[x,y] = 1
            Europe_mask[x,y] = 1
        else:
            Netherlands_mask[x,y] = np.nan

        if 'Luxembourg' in str(reverse_geocode.search(coordinates)):
            Luxembourg_mask[x,y] = 1
            Europe_mask[x,y] = 1
        else:
            Luxembourg_mask[x,y] = np.nan
            
        if 'Italy' in str(reverse_geocode.search(coordinates)):
            Italy_mask[x,y] = 1
        else:
            Italy_mask[x,y] = np.nan
                        

In [4]:
mask = {}
mask['1'] = UK_mask
mask['2'] = France_mask
mask['3'] = Germany_mask
mask['4'] = Switzerland_mask
mask['5'] = Austria_mask
mask['6'] = Belgium_mask
mask['7'] = Netherlands_mask
mask['8'] = Luxembourg_mask
mask['9'] = Italy_mask
mask['10'] = Europe_mask

## Wet Rankings All Years

In [5]:
#Country Ranking
PDSIavg = {}; index = {}
for i in range(1,11):
    country = owda.sel(time=slice(1300,2012))*np.tile(np.expand_dims(mask[str(i)],axis=2),[1,1,713])
    for y in range(0,88):
        country[:,y,:] = country[:,y,:]*np.cos(np.radians(owda.lat[y]))#area-weight
    PDSIavg[str(i)] = np.nanmean(np.nanmean(country,axis=0),axis=0)

    index[str(i)] = sorted(range(len(PDSIavg[str(i)])), key=lambda k: PDSIavg[str(i)][k] ,reverse=True)#PDSIavg is NOT sorted, only the index
    index[str(i)] = np.array(index[str(i)]) + 1300
    print(np.array(np.where(index[str(i)]==1317))+1)#insert year here, one is added to convert index to rank


[[296]]
[[378]]
[[201]]
[[189]]
[[407]]
[[416]]
[[342]]
[[336]]
[[615]]
[[344]]


In [6]:
#3 year ranking for 1314-1316
PDSI3avg = {}; index3 = {};
for i in range(1,11):
    PDSI3avg[str(i)] = np.array(pd.DataFrame(data=PDSIavg[str(i)]).rolling(3,min_periods=3).mean())[2::]
    index3[str(i)] = sorted(range(len(PDSI3avg[str(i)])), key=lambda k: PDSI3avg[str(i)][k] ,reverse=True)
    index3[str(i)] = np.array(index3[str(i)]) + 1300 + 2 #two is added to account for sliding three yr window
    print(np.array(np.where(index3[str(i)]==1316)) + 1) #one is added to convert index to rank

[[4]]
[[5]]
[[14]]
[[6]]
[[89]]
[[9]]
[[19]]
[[7]]
[[302]]
[[5]]


In [9]:
EuropePDSI = np.array(pd.DataFrame(data=PDSIavg['10']))
EuropePDSI3 = np.array(pd.DataFrame(data=PDSIavg['10']).rolling(3,min_periods=3).mean())
np.save('EuropePDSI',EuropePDSI)
np.save('EuropePDSI3',EuropePDSI3)



## Wet Rank, Famine Years Only

In [30]:
#1300 - 1710
years = {}
#UK
years['1'] = np.array([1315,1316,1317,1349,1350,1351,1437,1438,1555,1556,1557,1585,1586,1587,1594,1595,1596,1597,
                       1598,1622,1623,1698])
#France
years['2'] = np.array([1315,1316,1317,1347,1438,1573,1576,1585,1586,1587,1593,1598,1639,1653,1661,1676,1693,1694,
                         1709,1710])
#Germany
years['3'] = np.array([1315,1316,1317,1318,1437,1438,1439,1440,1516,1517,1518,1530,1531,1569,1570,1571,1572,1573,
                          1574,1594,1595,1596,1597,1598,1620,1621,1622,1623,1625,1626,1627,1628,1629,1630,1635,1636,
                          1651,1652,1691,1692,1693,1694,1695,1698,1699,1700,1701,1709,1710]) 
#Switzerland
years['4'] = np.array([1315,1316,1317,1318,1437,1438,1439,1440,1516,1517,1518,1530,1531,1569,1570,1571,1572,1573,
                          1574,1594,1595,1596,1597,1598,1620,1621,1622,1623,1625,1626,1627,1628,1629,1630,1635,1636,
                          1651,1652,1691,1692,1693,1694,1695,1698,1699,1700,1701,1709,1710]) 
#Austria
years['5'] = np.array([1315,1316,1317,1318,1437,1438,1439,1440,1516,1517,1518,1530,1531,1569,1570,1571,1572,1573,
                          1574,1594,1595,1596,1597,1598,1620,1621,1622,1623,1625,1626,1627,1628,1629,1630,1635,1636,
                          1651,1652,1691,1692,1693,1694,1695,1698,1699,1700,1701,1709,1710]) 
#Belgium
years['6'] = np.array([1315,1316,1317,1437,1438,1482,1483,1491,1492,1521,1522,1556,1557,1565,1566,1572,1573,1585,
                          1586,1587,1625,1626,1648,1649,1650,1651,1652,1661,1662,1674,1675,1676,1692,1693,1694,1698,
                          1699,1708,1709,1710])
#Netherlands
years['7'] = np.array([1315,1316,1317,1437,1438,1482,1483,1491,1492,1521,1522,1556,1557,1565,1566,1572,1573,1585,
                          1586,1587,1625,1626,1648,1649,1650,1651,1652,1661,1662,1674,1675,1676,1692,1693,1694,1698,
                          1699,1708,1709,1710])
#Luxembourg
years['8'] = np.array([1315,1316,1317,1437,1438,1482,1483,1491,1492,1521,1522,1556,1557,1565,1566,1572,1573,1585,
                          1586,1587,1625,1626,1648,1649,1650,1651,1652,1661,1662,1674,1675,1676,1692,1693,1694,1698,
                          1699,1708,1709,1710])
#Italy
years['9'] = np.array([1302,1303,1328,1329,1330,1339,1340,1346,1347,1374,1375,1410,1411,1412,1502,1503,1504,1505,
                       1527,1528,1529,1530,1539,1540,1569,1570,1571,1572,1586,1587,1590,1591,1592,1593,1600,1601,
                       1618,1619,1620,1621,1622,1628,1629,1648,1649,1678,1679,1693,1694,1695,1708,1709])
#Europe
years['10'] = np.array([1302,1303,1315,1316,1317,1328,1339,1340,1346,1347,1374,1375,1437,1438,1521,1522,1530,1556,
                        1557,1569,1570,1571,1572,1573,1574,1586,1587,1590,1591,1592,1593,1594,1595,1596,1597,1598,
                        1600,1601,1602,1603,1620,1621,1622,1623,1625,1626,1627,1628,1629,1630,1631,1647,1648,1649,
                        1650,1651,1652,1659,1660,1661,1662,1675,1676,1678,1679,1693,1694,1695,1696,1697,1698,1699,
                        1700,1708,1709,1710]) 

In [26]:
#1300 - 2012
years = {}
#UK
years['1'] = np.array([1315,1316,1317,1349,1350,1351,1437,1438,1555,1556,1557,1585,1586,1587,1594,1595,1596,1597,
                       1598,1622,1623,1698,1727,1728,1729,1730,1741,1742])
#France
years['2'] = np.array([1315,1316,1317,1347,1438,1573,1576,1585,1586,1587,1593,1598,1639,1653,1661,1676,1693,1694,
                         1709,1710,1719,1747,1788,1789])
#Germany
years['3'] = np.array([1315,1316,1317,1318,1437,1438,1439,1440,1516,1517,1518,1530,1531,1569,1570,1571,1572,1573,
                        1574,1594,1595,1596,1597,1598,1620,1621,1622,1623,1625,1626,1627,1628,1629,1630,1635,1636,
                        1651,1652,1691,1692,1693,1694,1695,1698,1699,1700,1701,1709,1710,1711,1712,1724,1725,1739,
                        1740,1741,1755,1756,1757,1758,1770,1771,1772,1787,1788,1789,1790,1793,1794,1795,1805,1815,
                        1816,1817,1845,1846,1847,1848,1916,1917,1918,1945,1946]) 
#Switzerland
years['4'] = np.array([1315,1316,1317,1318,1437,1438,1439,1440,1516,1517,1518,1530,1531,1569,1570,1571,1572,1573,
                          1574,1594,1595,1596,1597,1598,1620,1621,1622,1623,1625,1626,1627,1628,1629,1630,1635,1636,
                          1651,1652,1691,1692,1693,1694,1695,1698,1699,1700,1701,1709,1710,1711,1712,1724,1725,1739,
                        1740,1741,1755,1756,1757,1758,1770,1771,1772,1787,1788,1789,1790,1793,1794,1795,1805,1815,
                        1816,1817,1845,1846,1847,1848,1916,1917,1918,1945,1946]) 
#Austria
years['5'] = np.array([1315,1316,1317,1318,1437,1438,1439,1440,1516,1517,1518,1530,1531,1569,1570,1571,1572,1573,
                          1574,1594,1595,1596,1597,1598,1620,1621,1622,1623,1625,1626,1627,1628,1629,1630,1635,1636,
                          1651,1652,1691,1692,1693,1694,1695,1698,1699,1700,1701,1709,1710,1711,1712,1724,1725,1739,
                        1740,1741,1755,1756,1757,1758,1770,1771,1772,1787,1788,1789,1790,1793,1794,1795,1805,1815,
                        1816,1817,1845,1846,1847,1848,1916,1917,1918,1945,1946]) 
#Belgium
years['6'] = np.array([1315,1316,1317,1437,1438,1482,1483,1491,1492,1521,1522,1556,1557,1565,1566,1572,1573,1585,
                        1586,1587,1625,1626,1648,1649,1650,1651,1652,1661,1662,1674,1675,1676,1692,1693,1694,1698,
                        1699,1708,1709,1710,1740,1741,1794,1795,1816,1817,1845,1846,1847,1848,1849,1850,1916,1917,
                        1918,1944,1945])
#Netherlands
years['7'] = np.array([1315,1316,1317,1437,1438,1482,1483,1491,1492,1521,1522,1556,1557,1565,1566,1572,1573,1585,
                        1586,1587,1625,1626,1648,1649,1650,1651,1652,1661,1662,1674,1675,1676,1692,1693,1694,1698,
                        1699,1708,1709,1710,1740,1741,1794,1795,1816,1817,1845,1846,1847,1848,1849,1850,1916,1917,
                        1918,1944,1945])
#Luxembourg
years['8'] = np.array([1315,1316,1317,1437,1438,1482,1483,1491,1492,1521,1522,1556,1557,1565,1566,1572,1573,1585,
                        1586,1587,1625,1626,1648,1649,1650,1651,1652,1661,1662,1674,1675,1676,1692,1693,1694,1698,
                        1699,1708,1709,1710,1740,1741,1794,1795,1816,1817,1845,1846,1847,1848,1849,1850,1916,1917,
                        1918,1944,1945])
#Italy
years['9'] = np.array([1302,1303,1328,1329,1330,1339,1340,1346,1347,1374,1375,1410,1411,1412,1502,1503,1504,1505,
                       1527,1528,1529,1530,1539,1540,1569,1570,1571,1572,1586,1587,1590,1591,1592,1593,1600,1601,
                       1618,1619,1620,1621,1622,1628,1629,1648,1649,1678,1679,1693,1694,1695,1708,1709,1743,1744,
                       1745,1764,1765,1766,1767,1817])
#Europe
years['10'] = np.array([1302,1303,1315,1316,1317,1328,1339,1340,1346,1347,1374,1375,1437,1438,1521,1522,1530,1556,
                        1557,1569,1570,1571,1572,1573,1574,1586,1587,1590,1591,1592,1593,1594,1595,1596,1597,1598,
                        1600,1601,1602,1603,1620,1621,1622,1623,1625,1626,1627,1628,1629,1630,1631,1647,1648,1649,
                        1650,1651,1652,1659,1660,1661,1662,1675,1676,1678,1679,1693,1694,1695,1696,1697,1698,1699,
                        1700,1708,1709,1710,1719,1727,1728,1729,1730,1740,1741,1742,1743,1763,1764,1765,1771,1772,
                        1787,1788,1789,1794,1795,1803,1804,1805,1816,1817,1845,1846,1847,1848,1849,1850,1866,1867,
                        1868,1916,1917,1918,1944,1945,1946]) 

In [33]:
PDSIavg = {};
for i in range(1,11):
    PDSI = owda*np.tile(np.expand_dims(mask[str(i)],axis=2),[1,1,2013])
    for y in range(0,88):
        PDSI[:,y,:] = PDSI[:,y,:]*np.cos(np.radians(owda.lat[y]))#area-weight
    PDSIavg[str(i)] = np.nanmean(np.nanmean(PDSI,axis=0),axis=0)
    
    print(np.array(np.where(sorted(PDSIavg[str(i)][years[str(i)]],reverse=True)==PDSIavg[str(i)][1316]))+1)#rank


[[4]]
[[3]]
[[1]]
[[3]]
[[8]]
[[4]]
[[4]]
[[3]]
[[33]]
[[5]]


In [14]:
for i in range(1,11):
    print(np.mean(PDSIavg[str(i)][years[str(i)]]))

0.2244549940970231
0.017240368789682714
-0.35652987441053563
-0.23525366594800093
0.18410018992671767
-0.2853772308845427
-0.39257383061506995
-0.2938894921406021
0.05651930392156564
-0.1294819015828576


## Rank 1314 to years preceding famine

In [ ]:
#1300 - 1710
years_1314 = {}

#UK
years_1314['1'] = np.array([1314,1348,1436,1554,1584,1593,1621,1697])

#France
years_1314['2'] = np.array([1314,1346,1437,1572,1575,1584,1592,1597,1638,1652,1660,1675,1692,1708])

#Germany
years_1314['3'] = np.array([1314,1436,1515,1529,1568,1593,1619,1624,1634,1650,1690,1697,1708]) 

#Switzerland 
years_1314['4'] = np.array([1314,1436,1515,1529,1568,1593,1619,1624,1634,1650,1690,1697,1708]) 

#Austria
years_1314['5'] = np.array([1314,1436,1515,1529,1568,1593,1619,1624,1634,1650,1690,1697,1708]) 

#Belgium
years_1314['6'] = np.array([1314,1436,1481,1490,1520,1555,1564,1571,1584,1624,1647,1660,1673,1691,1697,1707])

#Netherlands
years_1314['7'] = np.array([1314,1436,1481,1490,1520,1555,1564,1571,1584,1624,1647,1660,1673,1691,1697,1707])

#Luxembourg
years_1314['8'] = np.array([1314,1436,1481,1490,1520,1555,1564,1571,1584,1624,1647,1660,1673,1691,1697,1707])

#Italy
years_1314['9'] = np.array([1301,1327,1338,1345,1373,1409,1501,1526,1538,1568,1585,1589,1599,1617,1627,1647,
                            1677,1692,1707])
#Europe
years_1314['10'] = np.array([1301,1314,1327,1338,1345,1373,1436,1520,1529,1555,1568,1585,1589,1599,1619,1624,
                             1646,1658,1674,1677,1692,1707])



In [12]:
#1300 - 2012
years_1314 = {}

#UK
years_1314['1'] = np.array([1314,1348,1436,1554,1584,1593,1621,1697,1726,1740])

#France
years_1314['2'] = np.array([1314,1346,1437,1572,1575,1584,1592,1597,1638,1652,1660,1675,1692,1708,1718,1746,1787])

#Germany
years_1314['3'] = np.array([1314,1436,1515,1529,1568,1593,1619,1624,1634,1650,1690,1697,1708,1723,1738,1754,1769,
                           1786,1792,1804,1814,1844,1915,1944]) 

#Switzerland 
years_1314['4'] = np.array([1314,1436,1515,1529,1568,1593,1619,1624,1634,1650,1690,1697,1708,1723,1738,1754,1769,
                           1786,1792,1804,1814,1844,1915,1944]) 

#Austria
years_1314['5'] = np.array([1314,1436,1515,1529,1568,1593,1619,1624,1634,1650,1690,1697,1708,1723,1738,1754,1769,
                           1786,1792,1804,1814,1844,1915,1944]) 

#Belgium
years_1314['6'] = np.array([1314,1436,1481,1490,1520,1555,1564,1571,1584,1624,1647,1660,1673,1691,1697,1707,
                           1739,1793,1815,1844,1915,1943])

#Netherlands
years_1314['7'] = np.array([1314,1436,1481,1490,1520,1555,1564,1571,1584,1624,1647,1660,1673,1691,1697,1707,
                           1739,1793,1815,1844,1915,1943])

#Luxembourg
years_1314['8'] = np.array([1314,1436,1481,1490,1520,1555,1564,1571,1584,1624,1647,1660,1673,1691,1697,1707,
                           1739,1793,1815,1844,1915,1943])

#Italy
years_1314['9'] = np.array([1301,1327,1338,1345,1373,1409,1501,1526,1538,1568,1585,1589,1599,1617,1627,1647,
                            1677,1692,1707,1742,1763,1816])

#Europe
years_1314['10'] = np.array([1301,1314,1327,1338,1345,1373,1436,1520,1529,1555,1568,1585,1589,1599,1619,1624,
                             1646,1658,1674,1677,1692,1707,1718,1726,1739,1762,1770,1786,1793,1802,1815,1844,
                            1865,1915,1943])



In [ ]:
PDSIavg = {};
for i in range(1,11):
    PDSI = owda*np.tile(np.expand_dims(mask[str(i)],axis=2),[1,1,2013])
    for y in range(0,88):
        PDSI[:,y,:] = PDSI[:,y,:]*np.cos(np.radians(owda.lat[y]))#area-weight
    PDSIavg[str(i)] = np.nanmean(np.nanmean(PDSI,axis=0),axis=0)
    
    print(np.array(np.where(sorted(PDSIavg[str(i)][years[str(i)]],reverse=True)==PDSIavg[str(i)][1314]))+1)#rank
